<a href="https://colab.research.google.com/github/donalglavin/donalglavin/blob/master/JupyterLab_SpinOff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupyter Lab... *spin off*
The purpose of this notebook is to spin off a *jupyter lab* instance via a public IP to provide greater control and access to scripting and data.

## Install Prerequsites.
As with most things, batteries are not included and there is some essential set up required to get the system up and running.

For this program the following items need to be installed.
  1. Installation of Jupyter Lab and Restart of runtime
  1. Installation of *pyngrok* to provide a means of tunnelling the local Jupyter Lab through to the public IP.


In [ ]:
!pip install pyngrok
!pip install jupyterlab
!pip install jupyter

Import the pyngrok library into the system context.

In [28]:
from pyngrok import ngrok

## Security Configuration.
With jupyter lab and pyngrok installed it's important to have some basic security settings configured to ensure we are not leaving ourselves exposed. These include:
1. The use of the ngrok authorisation token.
1. Configuration of a Jupyter Lab username and password.
1. The inclusion of a self signed certificate.

### Generate the self signed certificate.
Using openssl we can generate a new RSA certificate for the JupyterLab application.

The below code will create:
- /content/mycert.pem
- /content/mykey.key

In the root directory of this virtual machine.

In [ ]:
!openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout mykey.key -out mycert.pem

Confirm location of certificates.

In [ ]:
!ls -la /content/

### Define Jupyter Notebook Config.
The Jupyter notebook Configuration sets up the required variables for running the jupyter notebook.

Generate and overwrite the existing configuration file.

In [ ]:
!jupyter notebook --generate-config -y

Confim location of Jupyter Notebook Config.

In [ ]:
!ls -la /root/.jupyter/

Generating the modified config file.


In [ ]:
# The existing file contents.
!cat /root/.jupyter/jupyter_notebook_config.py

In [34]:
# Backup the config file.
!mv /root/.jupyter/jupyter_notebook_config.py /content/old_jupyter_config.py

In [46]:
jupyterConfig = ""
jupyterConfig = jupyterConfig + "c.NotebookApp.certfile = '/content/mycert.pem'" + "\n" # The full path to an SSL/TLS certificate file.
jupyterConfig = jupyterConfig + "c.NotebookApp.keyfile = '/content/mykey.key'" + "\n" # The full path to a private key file for usage with SSL/TLS.
jupyterConfig = jupyterConfig + "c.NotebookApp.ip = '*'" + "\n" # The IP address the notebook server will listen on.
jupyterConfig = jupyterConfig + "c.NotebookApp.open_browser = False" + "\n" # Dont open a browser straight away.
jupyterConfig = jupyterConfig + "c.NotebookApp.port = 80" + "\n" # The port the server will listen on.
jupyterConfig = jupyterConfig + "c.NotebookApp.allow_root = True" + "\n" # Allow root access (because VM is root and errors otherwise).
jupyterConfig = jupyterConfig + "c.NotebookApp.allow_remote_access = True" + "\n" # Allow remote access for login.
jupyterConfig = jupyterConfig + "c.NotebookApp.enable_mathjax = True" + "\n" # Allow the use of latex markup.
 #c.NotebookApp.token = ''
 #c.NotebookApp.password = ''

f = open("/root/.jupyter/jupyter_notebook_config.py", "w")
f.write(jupyterConfig)
f.close()


### Setup the NGROK authentication token.
Set the authorisation code via user input.

In [37]:
ngrok_auth_token = ""

In [ ]:
ngrok.set_auth_token(input())

In [38]:
ngrok.set_auth_token(ngrok_auth_token)

## Connect to Google Drive.
In order for this to really have any benefit the files I'll be working on will need to persist from edit to edit.

Hence why connecting to google drive is important.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls -la '/content/gdrive/MyDrive/Colab Notebooks/'

## Run Jupyter Lab & Ngrok

Firstly, set up and obtain the running ngrok public IP. This need to be executed before the jupyter lab application is started as it will block all other execution of code.

Present internal proxy port.

In [53]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://kojkl9ufuw9-496ff2e9c6d22116-8000-colab.googleusercontent.com/


Run ngrok.

In [51]:
# Open a HTTP tunnel on the default port 80. 
http_tunnel = ngrok.connect(port=80)
# Present a link to the tunnel. the tunnel.
http_tunnel

<NgrokTunnel: "http://3e50ee374a07.ngrok.io" -> "http://localhost:80">

Execute the Jupyter Lab application in the location of interest (colab notebook directory).

In [56]:
!jupyter lab '/content/gdrive/MyDrive/Colab Notebooks/' --port=8000 --allow-root --no-browser

[I 2021-07-21 12:18:09.932 ServerApp] jupyterlab | extension was successfully linked.
[W 2021-07-21 12:18:09.936 NotebookApp] 'allow_root' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2021-07-21 12:18:09.936 NotebookApp] 'allow_root' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2021-07-21 12:18:09.936 NotebookApp] 'port_retries' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2021-07-21 12:18:09.936 NotebookApp] 'contents_manager_class' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2021-07-21 12:18:09.936 NotebookApp] 'login_handler_class' has moved from NotebookApp to ServerApp. This config will be passed to ServerA

(use the token in the listed url as the token in the webpage, i.e. .../token?_\<copy this\>_.

In [50]:
# Kill ngrok (can only have one instance running at a time).
ngrok.kill()